In [1]:
from tensorflow.keras.layers import LSTM

class CustomLSTM(LSTM):
    def __init__(self, *args, **kwargs):
        kwargs.pop('time_major', None)  # Remove the 'time_major' argument if it exists
        super(CustomLSTM, self).__init__(*args, **kwargs)

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'CustomLSTM': CustomLSTM})

trained_model = load_model(r"C:\Users\pranali\Downloads\5g_classifier.h5", custom_objects={'LSTM': CustomLSTM})

trained_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_0 (CustomLSTM)             │ (None, 10, 120)        │        63,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (CustomLSTM)             │ (None, 10, 160)        │       179,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 160)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (CustomLSTM)             │ (None, 10, 120)        │       134,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_0 (Dense)                 │ (None, 56)             │        67,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         2,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OUT_multivariate_classifier     │ (None, 3)              │           147 │
│ (Dense)                         │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 450,573 (1.72 MB)

 Trainable params: 450,571 (1.72 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [2]:
import numpy as np
import tensorflow as tf

# Load your data
X_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Training\all_data_classifier_train_x.npy")
y_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Training\all_data_classifier_train_y.npy")
X_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Testing\all_data_classifier_test_x.npy")
y_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\Mature(1)\Mature-main\Datasets\Testing\all_data_classifier_test_y.npy")

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)



In [5]:
#integer quantization


import tensorflow as tf
import numpy as np

# Define the representative dataset generator for quantization calibration
def representative_data_gen():
    for input_value in X_train[:100]:  # Use a small sample of your training data
        input_value = np.expand_dims(input_value, axis=0).astype(np.float32)
        yield [input_value]

# Convert the model to TensorFlow Lite with full integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(trained_model)

# Set optimization to full integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide representative dataset for integer quantization
converter.representative_dataset = representative_data_gen

# Set the supported operations to enable both built-in ops and Select TF ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, 
                                       tf.lite.OpsSet.SELECT_TF_OPS]

# Set the input and output tensors to be int8 for full integer quantization
converter.inference_input_type = tf.int8  # or tf.uint8 based on your need
converter.inference_output_type = tf.int8  # or tf.uint8 based on your need

# Disable lowering of tensor list operations to avoid related issues
converter._experimental_lower_tensor_list_ops = False

# Enable resource variable support (required for LSTM layers)
converter.experimental_enable_resource_variables = True

# Convert the model
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmpw5od6bg3\assets


INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmpw5od6bg3\assets


Saved artifact at 'C:\Users\pranali\AppData\Local\Temp\tmpw5od6bg3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 11), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2306872878544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306872884496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306872884304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306872885072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874280656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874280848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874282384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874283344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874280464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2306874284496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  230687428

C:\Users\pranali\anaconda3\Lib\site-packages\tensorflow\lite\python\convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [6]:
# Save the TFLite model
with open('5g_classifier_integer.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [9]:
import tensorflow as tf
import numpy as np

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_path='5g_classifier_integer.tflite')
interpreter.allocate_tensors()

# Get details of input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
import numpy as np
import time

# Function to quantize input data
def quantize_input(input_data, input_scale, input_zero_point):
    return np.round(input_data / input_scale + input_zero_point).astype(np.int8)

# Initialize list to store predictions
predictions = []

# Get the quantization parameters for the input tensor
input_scale = input_details[0]['quantization'][0]
input_zero_point = input_details[0]['quantization'][1]

# Iterate over each sample in X_test
start_time = time.time()
for i in range(X_test.shape[0]):
    # Prepare the input data
    input_data = X_test[i].astype(np.float32)  # Keep as float32 for quantization
    input_data = quantize_input(input_data, input_scale, input_zero_point)
    input_data = np.expand_dims(input_data, axis=0)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(output_data)

end_time = time.time()
inference_time = (end_time - start_time)
print(f"Inference Time for full integer quantized model: {inference_time:.6f} seconds")

avg_inference_time = inference_time / len(X_test)
print(f"Avg Inference Time for full integer quantized model: {avg_inference_time:.6f} seconds")


Inference Time for full integer quantized model: 23.134471 seconds
Avg Inference Time for full integer quantized model: 0.003076 seconds


In [17]:
import os

# Get the size of the saved quantized TFLite model
model_size = os.path.getsize('5g_classifier_integer.tflite')
print(f"Quantized TFLite model size: {model_size / 1024:.2f} KB")  # Convert to KB



Quantized TFLite model size: 517.90 KB


In [15]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Function to quantize input data
def quantize_input(input_data, input_scale, input_zero_point):
    return np.round(input_data / input_scale + input_zero_point).astype(np.int8)

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get the quantization parameters for the input tensor
input_scale = input_details[0]['quantization'][0]
input_zero_point = input_details[0]['quantization'][1]

# Initialize list to store predictions and logits for loss calculation
predictions = []
logits = []

# Iterate over each sample in X_test
for i in range(X_test.shape[0]):
    #print(X_test[i])
    # Prepare the input data
    input_data = X_test[i].astype(np.float32)  # Keep as float32 for quantization
    input_data = quantize_input(input_data, input_scale, input_zero_point)
    input_data = np.expand_dims(input_data, axis=0)
    #print(input_data)
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor (classification result)
    output_data = interpreter.get_tensor(output_details[0]['index'])
    #print(output_data)
    # Cast the logits to float32 for loss calculation
    logits.append(output_data.astype(np.float32))  # Cast to float32
    predictions.append(np.argmax(output_data, axis=1))  # Store predicted class
    #print(predictions)
    #break
# Convert predictions and logits to numpy arrays and squeeze the shape if necessary
predictions = np.squeeze(np.array(predictions))
logits = np.squeeze(np.array(logits))

# For classification, get the predicted class and the true class
true_classes = np.argmax(y_test, axis=1)  # Assuming y_test is one-hot encoded

# Calculate the accuracy
accuracy = np.sum(predictions == true_classes) / len(true_classes)
print(f"Accuracy: {accuracy:.4f}")

# Calculate the loss using logits (after casting them to float32)
loss = tf.keras.losses.sparse_categorical_crossentropy(true_classes, logits, from_logits=True)
print(f"Loss: {np.mean(loss):.4f}")

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predictions)
print("\nConfusion Matrix:")
print(conf_matrix)

# Calculate and print the classification report
target_names = ["Low (0)", "Medium (1)", "High (2)"]
class_report = classification_report(true_classes, predictions, target_names=target_names)
print("\nClassification Report:")
print(class_report)


Accuracy: 0.7794
Loss: 41.9082

Confusion Matrix:
[[   0    0  373]
 [   0    0 1286]
 [   0    0 5861]]

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.00      0.00      0.00       373
  Medium (1)       0.00      0.00      0.00      1286
    High (2)       0.78      1.00      0.88      5861

    accuracy                           0.78      7520
   macro avg       0.26      0.33      0.29      7520
weighted avg       0.61      0.78      0.68      7520



C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
